In [47]:
import cups 
from PIL import Image

import sys
import time
import json
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler, FileCreatedEvent


In [48]:
conn = cups.Connection()
printers = conn.getPrinters()

print("Available Printers:")
for printer in printers:
	print(printer, printers[printer]["device-uri"])

Available Printers:
Brother_DCP_J987N dnssd://Brother%20DCP-J987N._ipp._tcp.local./?uuid=e3248000-80ce-11db-8000-b42200196a42
Canon_SELPHY_CP1500 ippusb://Canon%20SELPHY%20CP1500._ipp._tcp.local./?uuid=c624537c-b5e2-495e-a68f-5c625afd8142
HP_OfficeJet_Pro_6970 dnssd://HP%20OfficeJet%20Pro%206970%20%5BCB5C9F%5D._ipps._tcp.local./?uuid=5cf04ef8-f57d-f5b0-cef7-fe81cd0c7150
HP_OfficeJet_Pro_6970___Fax dnssd://HP%20OfficeJet%20Pro%206970%20%5BCB5C9F%5D._ipps._tcp.local./?uuid=5cf04ef8-f57d-f5b0-cef7-fe81cd0c7150
LLK_FX_Printer lpd://192.168.100.190/
OKI_MC562_E60FB9 dnssd://OKI-MC562-E60FB9._ipp._tcp.local./?uuid=0c3e2789-3e1d-49b3-b313-496a19343b0c


In [49]:

def resize_and_crop(img, target_width, target_height):
    # Calculate aspect ratio of desired size and original image
    target_ratio = target_width / target_height
    img_ratio = img.width / img.height

    # Resize the image to maintain the aspect ratio
    if target_ratio > img_ratio:
        # It means our target width is comparatively greater than the width of the image w.r.t their heights
        # So we set the width of the image to the target width and height is adjusted accordingly to maintain the aspect ratio
        img = img.resize((target_width, int(target_width / img_ratio)))
    else:
        # We set the height of the image to the target height and width is adjusted accordingly
        img = img.resize((int(target_height * img_ratio), target_height))

    # Centering and cropping the image
    left_margin = (img.width - target_width) / 2
    top_margin = (img.height - target_height) / 2
    right_margin = left_margin + target_width
    bottom_margin = top_margin + target_height

    img = img.crop((left_margin, top_margin, right_margin, bottom_margin))

    return img


In [50]:
# image_path = "/Users/tyoshida/Downloads/screenshot-1696711408854.png"
# image_path = "/Users/tyoshida/Downloads/DALLE3.jpg"

def print_image(image_path, printer_path="Canon_SELPHY_CP1500"):
    # Open the image and resize it
    img = Image.open(image_path, "r")
    img = img.convert("RGB")
    img = resize_and_crop(img, 3201, 2243)

    # Save the resized image to a temporary file
    temp_image_path = "/Users/tyoshida/Downloads/temp.jpg"
    img.save(temp_image_path)

    # Print the image
    conn.printFile(printer_path, temp_image_path, "Print Job", {})

    print("Print Finished")
    
# print_image(image_path)

In [51]:
class ScreenshotHandler(FileSystemEventHandler):
    def __init__(self):
        self.data = []
        self.filtered_data = []
        self.last_processed_path = None  # 追加

    def on_created(self, event: FileCreatedEvent):
        # print(f"Event triggered for: {event.src_path}")

        # 追加：もし前回と同じパスなら処理をスキップ
        if self.last_processed_path == event.src_path:
            return
        self.last_processed_path = event.src_path

        filename = event.src_path.split('/')[-1]
        if filename.endswith('.crdownload'):
            return
        if filename.startswith('screenshot') and filename.endswith('.png'):

            if isinstance(event, FileCreatedEvent):

                current_timestamp = time.time()
                entry = {
                    "path": event.src_path,
                    "timestamp": time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(current_timestamp))
                }
                self.data.append(entry)
                
                # Check if this is the first entry or if the timestamp difference from the last one is more than 3 seconds
                if (not self.filtered_data or 
                    (current_timestamp - time.mktime(time.strptime(self.filtered_data[-1]['timestamp'], '%Y-%m-%d %H:%M:%S')) > 10)):
                    if not any(d['path'] == event.src_path for d in self.filtered_data):
                        self.filtered_data.append(entry)
                        print_image(event.src_path)

                
                # print("Before printing filtered data")
                print("Filtered Data:", json.dumps(self.filtered_data, indent=4))


In [52]:

if __name__ == "__main__":
    path = "/Users/tyoshida/Downloads/"
    event_handler = ScreenshotHandler()
    observer = Observer()
    observer.schedule(event_handler, path, recursive=False)
    observer.start()
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()
